<a href="https://colab.research.google.com/github/MustaineWT/Yolo3v/blob/master/FinalMasivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 13939, done.
remote: Total 13939 (delta 0), reused 0 (delta 0), pack-reused 13939
Receiving objects: 100% (13939/13939), 12.53 MiB | 6.88 MiB/s, done.
Resolving deltas: 100% (9504/9504), done.


In [2]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [3]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# make darknet (build)
!make

In [ ]:
# get yolov3 pretrained coco dataset weights
!wget https://pjreddie.com/media/files/yolov3.weights

In [3]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)
      
def download(path):
  from google.colab import files
  files.download(path)

# use this to download a file drive  
def downloaddrive(each_image):
  import os  
  !cp "./predictions.jpg" "./results"
  #os.replace("./results/predictions.jpg",destino)
  os.rename (r'./results/predictions.jpg', r'./results/' + str (each_image))
  !cp "./results/{each_image}" "/content/drive/My Drive/yolo/resImagenes"

#proceso masivo drive to drive
def procesar():
  import os
  import cv2
  from os.path import basename
  
  items = os.listdir('/content/drive/My Drive/yolo/Imagenes')
  for each_image in items: 
    #each_image="5da130f55b2ff.jpg"
    eacj="'/content/drive/My Drive/yolo/Imagenes/'"+each_image
    !./darknet detector test cfg/botellas/obj.data cfg/botellas/obj.cfg cfg/botellas/obj_ajp.weights {str (eacj)} -thresh 0.3
    
    downloaddrive(each_image)

In [13]:
#!ls Copiar modelos cifrut
#!cp "/content/drive/My Drive/yolo/backup/yolov3_custom_1000.weights" "./cfg"
#!cp "/content/drive/My Drive/yolo/backup/yolov3_custom_2000.weights" "./cfg"
#!cp "/content/drive/My Drive/yolo/backup/yolov3_custom_final.weights" "./cfg"
#!cp "/content/drive/My Drive/yolo/backup/yolov3_custom_last.weights" "./cfg"
#!cp "/content/drive/My Drive/yolo/backup/yolov3_custom.cfg" "./cfg"
#!cp "/content/drive/My Drive/yolo/objetos/obj.names" "./data"
!cp "/content/drive/My Drive/yolo/objetos/obj.data" "./data"
#!cp "/content/drive/My Drive/yolo/objetos/generate_train.py" "./darknet"

In [10]:
#!ls Copiar modelos botellas
!cp "/content/drive/My Drive/yolo/AJE/obj_bebidas.weight" "./cfg/botellas"
!cp "/content/drive/My Drive/yolo/AJE/obj_ajp.weights" "./cfg/botellas"
!cp "/content/drive/My Drive/yolo/AJE/Data/obj-1.names" "./cfg/botellas"
!cp "/content/drive/My Drive/yolo/AJE/Data/obj.data" "./cfg/botellas"
!cp "/content/drive/My Drive/yolo/AJE/CFG/obj.cfg" "./cfg/botellas"

In [15]:
!python "generate_train.py"

In [ ]:
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov3_custom.cfg
!sed -i 's/subdivisions=64/subdivisions=1/' yolov3_custom.cfg
%cd ..
!darknet detector train data/obj.data cfg/yolov3_custom.cfg cfg/yolov3_custom_last.weights -dont_show
%cd cfg
!sed -i 's/batch=1/batch=64/' yolov3_custom.cfg
!sed -i 's/subdivisions=1/subdivisions=64/' yolov3_custom.cfg
%cd ..

In [ ]:
%cd cfg/botellas
!ls
!sed -i 's/batch=64/batch=1/' obj.cfg
!sed -i 's/subdivisions=64/subdivisions=1/' obj.cfg
%cd ..
%cd ..

In [ ]:
!./darknet detector train cfg/botellas/obj.data cfg/botellas/obj.cfg cfg/botellas/obj_ajp.weights -dont_show

In [ ]:
%cd cfg/botellas
!sed -i 's/batch=1/batch=64/' obj.cfg
!sed -i 's/subdivisions=1/subdivisions=64/' obj.cfg
%cd ..
%cd ..

In [ ]:
procesar()
#each_image="5da130f55b2ff.jpg"
#eacj="'/content/drive/My Drive/yolo/Imagenes/'"+each_image

#!./darknet detector test cfg/botellas/obj.data cfg/botellas/obj.cfg cfg/botellas/obj_ajp.weights {str (eacj)} -thresh 0.3
#print(eacj)
#/content/drive/My Drive/yolo/Imagenes/5da130f55b2ff.jpg
#imShow('predictions.jpg')
#import os
#import cv2
#from os.path import basename
#va=0
#items = os.listdir('/content/drive/My Drive/yolo/Imagenes')
#for each_image in items: 
#  va=va+1
#  print(each_image+str(va))
#%cd /content/darknet/